In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import sys
import re
import os
from pathlib import Path
from functools import partial
from dotenv import load_dotenv

load_dotenv()

import blpapi
import clarion
import ipywidgets as w
import ipyvuetify as v
import numpy as np
import pandas as pd
import QuantLib as ql
from IPython.display import display
from ipydatagrid import DataGrid, TextRenderer, BarRenderer, Expr
from bqplot import LinearScale, ColorScale, OrdinalColorScale, OrdinalScale

# import clarion as cl
import bhutils
from bhutils import bhutilspy as bp



In [ ]:
# import clarion

In [3]:
clarion.login()

Please enter your OTP and hit enter:  ········


2022-12-06 12:53:34,121 - Authenticating...
2022-12-06 12:53:35,288 - Successfully authenticated


In [ ]:
os.getenv('https_proxy')

In [ ]:
clarion.metrics()

In [ ]:
strategy_map = {
    'MM:EUR FLY SPRD': 'eurfly',
    'MM:EUR OPEN OPT': 'europt',
    'MM:EUR SHORT CU': 'eursht',
    'MM:EURUSD BOX': 'eurbox',
    'MM:USD OPEN OPT': 'usdopt',
    'MM:USD SHORT CU': 'usdsht'
}

In [ ]:
bpv = 10000
exchanges = [
    'ICE',
    'CME',
    'CBOT'
]

def today():
    return pd.Timestamp.today().floor('d')


def parse_clipboard(cols=None):
    cols = cols or [
        ('Trade Details', 'Strategy'),
        ('Trade Details', 'Trade Type'),
        ('Trade Details', 'Ccy'), 
        ('Trade Details', 'Instrument'), 
        ('Trade Details', 'Expiry/ Tenor'),
        ('Trade Details', 'Start Bumped'),
        ('Trade Details', 'Maturity Bumped'),
        ('Trade Details', 'Notional'),
        ('$ Risk', 'Delta'),
    ]
    
    date_cols = [
        ('Trade Details', 'Start Bumped'),
        ('Trade Details', 'Maturity Bumped'),
    ]

    num_cols = [
        ('Trade Details', 'Notional'),
        ('$ Risk', 'Delta')
    ]
    
    # data = pd.read_clipboard(sep='\t', header=[1], parse_dates=date_cols)
    
    
    data = pd.read_clipboard(sep='\t', header=[0, 1])

    idx = data.columns.to_frame().reset_index(drop=True)
    idx.loc[idx[0].str.contains('Unnamed'), 0] = np.nan
    idx[0] = idx[0].ffill()
    data = data.set_axis(pd.MultiIndex.from_frame(idx), axis=1)

    for col in num_cols:
        data[col] = data[col].str.replace(',', '').astype(float)
        
    for col in date_cols:
        data[col] = pd.to_datetime(data[col])
        
    return data[cols].droplevel(0, axis=1).dropna(subset=['Strategy', 'Instrument'])


def parse_swap_period(start_date=None, end_date=None, age=None):
    end_date = end_date or (today() + parse_offset(age) if age else today())
    start = np.abs(int(np.round(((start_date - end_date).days / 30.5))))
    if not start % 12:
        return f'{start // 12}y'
    else:
        return f"{start}m"
    

def parse_swap_periods(start_date, end_date, sep='x', age=None):
    start = parse_swap_period(start_date=start_date, age=age)
    tail = parse_swap_period(start_date=start_date, end_date=end_date)
    return f"{start}{sep}{tail}"


def parse_instrument(instrument):
    for ex in exchanges:
        instrument = instrument.replace(ex, '')
    return instrument.strip()


def parse_curve(instrument, currency, instrument_type=None):
    instrument = parse_instrument(instrument)
    if any(s in instrument for s in ['SOFR', 'ESTR']):
        curve = instrument.split(' ')[0].lower()
        curve = f"{currency.lower()}.{curve}"
    elif 'FEDFUNDS' in instrument:
        curve = f"{currency.lower()}.ois"
    elif any(s in instrument for s in ['EURIBOR', 'LIBOR']):
        split = instrument.split(' ')
        if split[-1][-1] in ['M', 'Y']:
            tenor = split[-1]
            if currency == 'USD' and tenor == '3M':
                suffix = ''
            elif currency == 'EUR' and tenor == '6M':
                suffix = ''
            else:
                suffix = f".{tenor.lower()}"
        else:  # maybe this block is never reached
            suffix = ''

        curve = f"{currency.lower()}{suffix}"
    elif instrument == 'US TY':
        curve = 'usd.sofr'
    return curve


def get_swap_string(
    instrument,
    currency,
    start_date, 
    end_date, 
    age=None,
    instrument_type=None,
):
    curve = parse_curve(instrument=instrument, currency=currency, instrument_type=instrument_type)
    swap_periods = parse_swap_periods(start_date=start_date, end_date=end_date, age=age)
    return f"[s {curve} {swap_periods}]"

In [ ]:
d = parse_clipboard()

In [ ]:
delta_tol = 100
delta_mul = 1
age = '3m'

In [ ]:
d = d.loc[~d['Delta'].between(-delta_tol, delta_tol), :]
d['Delta'] = d['Delta'] * delta_mul

dg = d.groupby(
    ['Strategy', 'Trade Type', 'Ccy', 'Instrument', 'Expiry/ Tenor', 'Start Bumped', 'Maturity Bumped'],
    as_index=False
).sum()

dg = dg.loc[~dg['Delta'].between(-delta_tol, delta_tol), :]

dg['curve'] = dg.apply(lambda row: parse_curve(
    instrument=row['Instrument'],
    currency=row['Ccy'], 
), axis=1)

dg['period'] = dg.apply(lambda row: parse_swap_periods(
    start_date=row['Start Bumped'], 
    end_date=row['Maturity Bumped'],
), axis=1)

dg['period_aged'] = dg.apply(lambda row: parse_swap_periods(
    start_date=row['Start Bumped'], 
    end_date=row['Maturity Bumped'],
    age=age
), axis=1)

dg['string'] = dg.apply(lambda row: get_swap_string(
    instrument=row['Instrument'],
    currency=row['Ccy'], 
    start_date=row['Start Bumped'], 
    end_date=row['Maturity Bumped'],
), axis=1)

dg['string_aged'] = dg.apply(lambda row: get_swap_string(
    instrument=row['Instrument'],
    currency=row['Ccy'], 
    start_date=row['Start Bumped'], 
    end_date=row['Maturity Bumped'],
    age=age,
), axis=1)

In [ ]:
dg_ = dg.groupby(['Strategy', 'string', 'string_aged'], as_index=False).sum() 

In [ ]:
def get_history_plotter_strings(strategy_map=None, delta_tol=100, delta_mul=1, age='3m'):
    strategy_map = strategy_map or {}
    d = parse_clipboard()
    d = d.loc[~d['Delta'].between(-delta_tol, delta_tol), :]
    d['Delta'] = d['Delta'] * delta_mul
    
    dg = d.groupby(
        ['Strategy', 'Trade Type', 'Ccy', 'Instrument', 'Expiry/ Tenor', 'Start Bumped', 'Maturity Bumped'],
        as_index=False
    ).sum()
    
    dg['curve'] = dg.apply(lambda row: parse_curve(
        instrument=row['Instrument'],
        currency=row['Ccy'], 
    ), axis=1)

    dg['period'] = dg.apply(lambda row: parse_swap_periods(
        start_date=row['Start Bumped'], 
        end_date=row['Maturity Bumped'],
    ), axis=1)
    
    dg['period_aged'] = dg.apply(lambda row: parse_swap_periods(
        start_date=row['Start Bumped'], 
        end_date=row['Maturity Bumped'],
        age=age
    ), axis=1)

    dg['string'] = dg.apply(lambda row: get_swap_string(
        instrument=row['Instrument'],
        currency=row['Ccy'], 
        start_date=row['Start Bumped'], 
        end_date=row['Maturity Bumped'],
    ), axis=1)

    dg['string_aged'] = dg.apply(lambda row: get_swap_string(
        instrument=row['Instrument'],
        currency=row['Ccy'], 
        start_date=row['Start Bumped'], 
        end_date=row['Maturity Bumped'],
        age=age,
    ), axis=1)
    
    parsed = {}
    for index, group in dg.groupby('Strategy'):
        s = (group['Delta'] / bpv).round(1).apply(lambda x: format(x, '+.2f'))  + '*' + group['string']
        s = f"{' '.join(s)}"
        parsed[strategy_map.get(index, index)] = s

    parsed = pd.Series(parsed, dtype=object)
    
    parsed_aged = {}
    for index, group in dg.groupby('Strategy'):
        s = (group['Delta'] / bpv).round(1).apply(lambda x: format(x, '+.2f'))  + '*' + group['string_aged']
        s = f"{' '.join(s)}"
        parsed[f"{strategy_map.get(index, index)}A"] = s

    parsed_aged = pd.Series(parsed_aged, dtype=object)
    parsed_all = pd.concat([parsed, parsed_aged])
    parsed_all.to_clipboard(header=False)
    return d, dg, parsed_all

In [ ]:
from api.tools import get_history_plotter_strings

In [ ]:
r = get_history_plotter_strings(strategy_map=strategy_map, delta_mul=-1)

In [ ]:
ptf = pd.Series(
    [
        ' + '.join(r[-1].index[:len(r[-1])//2]),
        ' + '.join(r[-1].index[len(r[-1])//2:])
    ],
    index=['ptf', 'ptfA']
)

In [ ]:
def get_history_plotter_strings(strategy_map=None, delta_tol=100, delta_mul=1, age='3m'):
    strategy_map = strategy_map or {}
    d = parse_clipboard()
    d = d.loc[~d['Delta'].between(-delta_tol, delta_tol), :]
    d['Delta'] = d['Delta'] * delta_mul
    
    dg = d.groupby(
        ['Strategy', 'Trade Type', 'Ccy', 'Instrument', 'Expiry/ Tenor', 'Start Bumped', 'Maturity Bumped'],
        as_index=False
    ).sum()
    
    dg['curve'] = dg.apply(lambda row: parse_curve(
        instrument=row['Instrument'],
        currency=row['Ccy'], 
    ), axis=1)

    dg['period'] = dg.apply(lambda row: parse_swap_periods(
        start_date=row['Start Bumped'], 
        end_date=row['Maturity Bumped'],
    ), axis=1)
    
    dg['period_aged'] = dg.apply(lambda row: parse_swap_periods(
        start_date=row['Start Bumped'], 
        end_date=row['Maturity Bumped'],
        age=age
    ), axis=1)

    dg['string'] = dg.apply(lambda row: get_swap_string(
        instrument=row['Instrument'],
        currency=row['Ccy'], 
        start_date=row['Start Bumped'], 
        end_date=row['Maturity Bumped'],
    ), axis=1)

    dg['string_aged'] = dg.apply(lambda row: get_swap_string(
        instrument=row['Instrument'],
        currency=row['Ccy'], 
        start_date=row['Start Bumped'], 
        end_date=row['Maturity Bumped'],
        age=age,
    ), axis=1)
    
    parsed = {}
    for index, group in dg.groupby('Strategy'):
        s = (group['Delta'] / bpv).round(1).apply(lambda x: format(x, '+.2f'))  + '*' + group['string']
        s = f"{' '.join(s)}"
        parsed[strategy_map.get(index, index)] = s

    parsed = pd.Series(parsed, dtype=object)
    
    parsed_aged = {}
    for index, group in dg.groupby('Strategy'):
        s = (group['Delta'] / bpv).round(1).apply(lambda x: format(x, '+.2f'))  + '*' + group['string_aged']
        s = f"{' '.join(s)}"
        parsed[f"{strategy_map.get(index, index)}A"] = s

    parsed_aged = pd.Series(parsed_aged, dtype=object)
    parsed_all = pd.concat([parsed, parsed_aged])
    parsed_all.to_clipboard(header=False)
    return d, dg, parsed_all

In [ ]:
from api.tools import get_history_plotter_strings

In [ ]:
r = get_history_plotter_strings(strategy_map=strategy_map, delta_mul=-1)

In [ ]:
ptf = pd.Series(
    [
        ' + '.join(r[-1].index[:len(r[-1])//2]),
        ' + '.join(r[-1].index[len(r[-1])//2:])
    ],
    index=['ptf', 'ptfA']
)

In [ ]:
ptf.to_clipboard(header=False)

In [ ]:
from api.history_plotter import get_expressions_data

In [ ]:
from api.ezutils import bh_get_swap_ts

In [ ]:
bh_get_swap_ts(expr='s usd.sofr 2mx79m')

In [ ]:
df = get_expressions_data(expressions)

In [ ]:
df.plot(backend='plotly', height=700)

In [ ]:
from api.ezutils import bh_get_forward_swap_ts

In [ ]:
start_date = pd.Timestamp('2022-05-26')
end_date = pd.Timestamp('2022-07-26')
curve_name = 'USD'
forward_start = "29M"
tenor = "3M"

In [ ]:
start_date = pd.Timestamp('2022-05-26')
end_date = pd.Timestamp('2022-07-26')
curve_name = 'eur'
forward_start = "7y"
tenor = "3y"

In [ ]:
ts = bh_get_forward_swap_ts(curve_name, forward_start, tenor, start_date, end_date)

In [ ]:
ts

In [ ]:
ts.plot(backend='plotly')

In [ ]:
dbh = bhup.bhReadMergedTsCurves(
    _dbhandle='dbtest',
    _tscrvhandle='crv1',
    _startdate=to_xl_date(start_date),
    _enddate=to_xl_date(end_date),
    _market="COB_COMPLETE",
    _curvename=f"{curve_name.upper()}.STD",
    _source="CRV",
    _category="O",
    _class="BHCURVE",
    _curvesdbname="Curves",
    _curvesdbserver="SQLP1-A",
    _histdbname="Histories",
    _histdbserver="SQLP1-A",
)

In [ ]:
calendar = bhup.bhDbLookUp(
    calendars,
    curve_name[:3].upper(),
    "Currency",
    "Calendar",
    0
)

In [ ]:
crvh = bhup.bhDbGetColValues(dbh, "TsCurve")[0][0]

In [ ]:
_type = "sw" if (start_date + parse_offset(tenor) - start_date).days > 360 else "dp"

rates_ts = bhup.bhTsRate(
    'ts1',
    _crvstsid=crvh,
    _efforfwd=forward_start,
    _mtyortenor=tenor,
    _type=_type,
    _ccy=f"{curve_name[:3].upper()}.STD",
    _mktdefs=mkt_defs,
    _calprovider=calendar,
    _leavemtyflag=1
)

ts = pd.Series(bhup.bhTsGet(rates_ts, "v").squeeze(axis=1), index=bhup.bhTsGet(rates_ts, "d").squeeze(axis=1)).plot(backend="plotly")

In [ ]:
pd.Series(bhup.bhTsGet(rates_ts, "v").squeeze(axis=1), index=bhup.bhTsGet(rates_ts, "d").squeeze(axis=1)).plot(backend="plotly")

In [ ]:
pd.DataFrame(np.concatenate([bhup.bhTsGet(rates_ts, "d"), bhup.bhTsGet(rates_ts, "v")], axis=1))

In [ ]:
ts = 

In [ ]:
bhup.bhTsGet(rates_ts, "d")

#### Settings

In [ ]:
dt_fmt = '%Y-%m-%d'
bbg_dt_fmt = '%Y%m%d'

#### Bloomberg Connection

In [ ]:
bq = BlpQuery(timeout=50000).start()
self = bq

In [ ]:
dl_btn = v.Btn(
    fab=True,
    # dark=True,
    small=True,
    plain=True,
    class_='mx-0',
    children=[v.Icon(children=['mdi-download'])]
)

refresh_btn = v.Btn(
    fab=True,
    # dark=True,
    small=True,
    plain=True,
    class_='mx-2',
    children=[v.Icon(children=['mdi-sync'])]
)

pcs_tf = v.TextField(
    v_model=None,
    dense=True,
    label='pcs',
    clearable=True
)

In [ ]:
class Comp(View):
    
    def make_widgets(self):
        paste_btn = v.Btn(
            fab=True,
            # dark=True,
            small=True,
            plain=True,
            class_='mx-0',
            children=[v.Icon(children=['mdi-content-paste'])]
        )

        refresh_btn = v.Btn(
            fab=True,
            # dark=True,
            small=True,
            plain=True,
            class_='mx-2',
            children=[v.Icon(children=['mdi-sync'])]
        )

        pcs_tf = v.TextField(
            v_model='BGN',
            outlined=True,
            dense=True,
            label='pcs',
            clearable=True
        )
        self.paste_btn = dl_btn
        self.refresh_btn = refresh_btn
        self.pcs_tf = pcs_tf
        
        self.out = w.Output()
    
    def make_view(self):
        
        paste_btn = self.paste_btn
        refresh_btn = self.refresh_btn
        pcs_tf = self.pcs_tf
        out = self.out
        
        param_box = v.Container(
            children=[
                v.Row(
                    children=[
                        v.Col(
                            cols=2,
                            children=[pcs_tf],
                            class_="my-0 py-0"
                        ),
                        v.Col(
                            cols=2,
                            children=[paste_btn, refresh_btn],
                            class_="my-0 py-0"
                        ),

                    ],
                    align_content='center',
                ),
            ]
        )
        
        view = w.VBox(
            children=[
                param_box,
                out,
            ]
        )

        self.param_box = param_box
        self.view = view
    
    def link(self):
        pass

In [ ]:
self = Comp()
self

In [ ]:
def on_click_paste(widget, event, data, self):
    data = pd.read_clipboard(sep='\t', header=[1])
    data['Notional'] = data['Notional'].str.replace(',', '').astype(float)
    cols = [
        'Issuer', 
        'Trade Type',
        'Description',
        'Ccy',
        'Expiry/ Maturity',
        'ISIN',
        'Total',
        'Notional',
        'Quoted Price',
    ]
    data_ = data[cols].dropna(subset='ISIN')
    data_ = data_.drop('Notional', axis=1)\
        .drop_duplicates(subset='ISIN')\
        .merge(data_.groupby('ISIN', as_index=False, sort=False)['Notional'].sum(), on='ISIN')
    data_ = data_[[*data_.columns[:-2], data_.columns[-1], data_.columns[-2]]]
    data_ = data_.loc[data_['Notional'] != 0, :]
    
    self.data = data
    self.data_ = data_

In [ ]:
out = self.out

In [ ]:
self.paste_btn.on_event(
    'click', 
    partial(on_click_paste, self=self)
)

In [ ]:
self.refresh_btn.on_event(
    'click', 
    partial(
        out.capture(on_click_refresh_grid), 
        self=self
    )
)

In [ ]:
data = pd.read_clipboard(sep='\t', header=[1])
data['Notional'] = data['Notional'].str.replace(',', '').astype(float)

In [ ]:
cols = [
    'Issuer', 
    'Trade Type',
    'Description',
    'Ccy',
    'Expiry/ Maturity',
    'ISIN',
    'Total',
    'Notional',
    'Quoted Price',
]

In [ ]:
data_ = data[cols].dropna(subset='ISIN')
data_ = data_.drop('Notional', axis=1)\
    .drop_duplicates(subset='ISIN')\
    .merge(data_.groupby('ISIN', as_index=False, sort=False)['Notional'].sum(), on='ISIN')
data_ = data_[[*data_.columns[:-2], data_.columns[-1], data_.columns[-2]]]
data_ = data_.loc[data_['Notional'] != 0, :]

In [ ]:
def get_securities(securities, id_type='isin', pcs=None, sep=None):
    if pcs:
        if id_type.lower() == 'isin':
            sep = '@'
        else: 
            sep = ' '
        securities = add_pcs(securities, source=pcs, sep=sep)
    
    return [f'/{id_type}/{sec}' for sec in securities]

In [ ]:
pcs = 'BGN'
sep = '@'

In [ ]:
securities = get_securities(data_['ISIN'], pcs=pcs)

In [ ]:
def bbg_comp(data_, pcs=None):
    securities = get_securities(data_['ISIN'], pcs=pcs)
    fields = [
        'px_last',
    ]
    
    bbg_data = bq.bdp(
        securities=securities,
        fields=fields
    )
    bbg_data['security'] = bbg_data['security'].replace(data_['ISIN'].set_axis(securities))
    comp_data = data_.merge(bbg_data, how='left', left_on='ISIN', right_on='security').drop('security', axis=1)
    comp_data['price_diff'] = (comp_data['Quoted Price'] - comp_data['px_last']).abs()
    comp_data['notional_diff'] = (comp_data['price_diff'] * comp_data['Notional']).abs()
    comp_data = comp_data.sort_values('price_diff', ascending=False)
    
    return comp_data

In [ ]:
def on_click_refresh_grid(widget, event, data, self):
    data_ = self.data_
    pcs = self.pcs_tf.v_model
    comp_data = bbg_comp(data_, pcs)
    self.comp_data = comp_data
    dg = make_datagrid(comp_data)
    self.dg = dg
    display(dg)

In [ ]:
fields = [
    'px_last',
]

In [ ]:
bbg_data = bq.bdp(
    securities=securities,
    fields=fields
)

In [ ]:
bbg_data['security'] = bbg_data['security'].replace(data_['ISIN'].set_axis(securities))

In [ ]:
comp_data = data_.merge(bbg_data, how='left', left_on='ISIN', right_on='security').drop('security', axis=1)

In [ ]:
comp_data['price_diff'] = (comp_data['Quoted Price'] - comp_data['px_last']).abs()
comp_data['notional_diff'] = (comp_data['price_diff'] * comp_data['Notional']).abs()
comp_data = comp_data.sort_values('price_diff', ascending=False)

In [ ]:
def millify(cell):
    n = abs(float(cell.value))
    
    if n < 1000:
        return format(cell.value, '.3f') 
    elif n < 1000000:
        return format(cell.value / 1000, '.3f') + 'k'
    elif n < 1000000000:
        return format(cell.value / 1000000, '.3f') + 'm'
    elif n < 1000000000000:
        return format(cell.value / 1000000000, '.3f') + 'b'
    else:
        return format(cell.value / 1000000000000, '.3f') + 't'
    
def text_color(cell):
    pass

In [ ]:
def make_datagrid(comp_data):
    renderers = {

        'Notional': TextRenderer(
            text_value=Expr(millify),
            text_color=Expr('"red" if cell.value < 0 else default_value')
        ),
        'notional_diff': BarRenderer(
            text_value=Expr(millify),
            horizontal_alignment="center",
            bar_color=ColorScale(min=comp_data['notional_diff'].min(), max=comp_data['notional_diff'].max()),
            bar_value=LinearScale(min=comp_data['notional_diff'].min(), max=comp_data['notional_diff'].max())
        ),
        'price_diff': BarRenderer(
            text_value=Expr("format(cell.value, '.2f')"),
            horizontal_alignment="center",
            bar_color=ColorScale(min=comp_data['price_diff'].min(), max=comp_data['price_diff'].max()),
            bar_value=LinearScale(min=comp_data['price_diff'].min(), max=comp_data['price_diff'].max())
        )
    }
    
    dg = DataGrid(comp_data, renderers=renderers)
    dg.auto_fit_columns = True
    return dg


In [ ]:
renderers = {

    'Notional': TextRenderer(
        text_value=Expr(millify),
        text_color=Expr('"red" if cell.value < 0 else default_value')
    ),
    'notional_diff': BarRenderer(
        text_value=Expr(millify),
        horizontal_alignment="center",
        bar_color=ColorScale(min=comp_data['notional_diff'].min(), max=comp_data['notional_diff'].max()),
        bar_value=LinearScale(min=comp_data['notional_diff'].min(), max=comp_data['notional_diff'].max())
    ),
    'price_diff': BarRenderer(
        text_value=Expr("format(cell.value, '.2f')"),
        horizontal_alignment="center",
        bar_color=ColorScale(min=comp_data['price_diff'].min(), max=comp_data['price_diff'].max()),
        bar_value=LinearScale(min=comp_data['price_diff'].min(), max=comp_data['price_diff'].max())
    )
}

In [ ]:
dg = DataGrid(comp_data, renderers=renderers)
dg.auto_fit_columns = True

In [ ]:
dg